In [1]:
####財務パネルデータを作成####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
#データの読み込み
company = pd.read_csv("D:/Statistics/data/finance_data/stocklist.csv")
finance_data1 = pd.read_csv("D:/Statistics/data/finance_data/finance_data.csv")
finance_data2 = pd.read_csv("D:/Statistics/data/finance_data/20190605_finance_data.csv")

#読み込んだデータをクレンジング
finance_data2 = finance_data2[["証券コード", "企業名", "会計基準", "連結個別", "決算期", "決算期間", "売上高", "営業利益", "純利益"]]
finance_data2.columns = ["ticker", "company", "accounting", "single", "financial_period", "quarter", 
                         "sales", "operating_income", "net_income"]
finance_data2["industry"] = pd.merge(finance_data2[["ticker"]], company, left_on="ticker", right_on="銘柄コード", how="left")["業種分類"]

#財務データを百万円単位にする
finance_data2["sales"] = np.array(finance_data2["sales"].str.replace("\,", ""), dtype="float") / 1000000.0
finance_data2["operating_income"] = np.array(finance_data2["operating_income"].str.replace("\,", ""), dtype="float") / 1000000.0
finance_data2["net_income"] = np.array(finance_data2["net_income"].str.replace("\,", ""), dtype="float") / 1000000.0

In [3]:
##finance_data1をクレンジング
#会計期間を分割
n1 = finance_data1.shape[0]
quarter_split = finance_data1["quarter"].str.split(" ")
fixed = np.array(np.repeat("", n1), dtype="object")
year1 = np.repeat(0, n1)
quarter1 = np.array(np.repeat("", n1), dtype="object")
for i in range(n1):
    if len(quarter_split[i])==1:
        continue
    if len(quarter_split[i])==3:
        year1[i] = np.array(quarter_split[i][1], dtype="int")
        quarter1[i] = quarter_split[i][2]
        fixed[i] = quarter_split[i][0]
    else:
        year1[i] = np.array(quarter_split[i][0], dtype="int")
        quarter1[i] = quarter_split[i][1]
quarter0 = quarter1.copy()

In [4]:
#四半期の名称を変更
index = np.where(pd.Series(quarter1).str.contains("通期"))[0]
quarter0[index] = "通期"
quarter0[quarter0=="第1四"] = "第1四半期"
quarter0[quarter0=="第2四"] = "第2四半期"
quarter0[quarter0=="第3四"] = "第3四半期"
quarter0[quarter0=="第4四"] = "第4四半期"

In [5]:
#会計期間を格納
finance_df1 = finance_data1[["ticker", "company", "industry", "sales", "operating_income", "net_income"]]
finance_df1["year"] = year1
finance_df1["quarter"] = quarter0

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
#業績予想をユニークにする
ticker = finance_df1[["ticker"]]
ticker["fixed"] = fixed
index1 = np.where(ticker["fixed"]!="")[0]
index2 = np.where(ticker["fixed"]=="")[0]
index = np.sort(np.append(index1[np.where(ticker["ticker"].iloc[index1].duplicated()==False)[0]], index2))
finance_df1 = finance_df1.iloc[index]
finance_df1.index = np.arange(finance_df1.shape[0])

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
#決算基準を結合
accounting = finance_data2[["ticker", "accounting", "single"]].iloc[np.where(finance_data2[["ticker"]].duplicated()==False)[0]]
accounting.index = np.arange(accounting.shape[0])
finance_df1 = pd.merge(finance_df1, accounting, on="ticker", how="left")
finance_df1 = finance_df1.iloc[np.where(finance_df1["year"]!=2017)[0]]
finance_df1.index = np.arange(finance_df1.shape[0])

In [8]:
##finance_data2をクレンジング
#会計期間を分割
quarter2 = np.array(finance_data2["financial_period"].str[:4], dtype="int")
finance_df2 = finance_data2[["ticker", "company", "industry", "accounting", "single", "quarter", "sales", "operating_income", "net_income"]]
finance_df2["year"] = quarter2

In [9]:
#決算月を結合
period1 = finance_data1.iloc[np.where(finance_data1["quarter"].str.contains("[0-9]月通期"))[0]][["ticker", "quarter"]]
period1["period"] = period1["quarter"].str[5:7]
period1.index = np.arange(period1.shape[0])
period2 = finance_data2[["ticker", "financial_period"]]
period2["period"] = finance_data2["financial_period"].str[5:7]
period = pd.concat((period1[["ticker", "period"]] , period2[["ticker", "period"]]), axis=0)
period.index = np.arange(period.shape[0])
period = period.iloc[np.where(period["ticker"].duplicated()==False)[0]]
period.index = np.arange(period.shape[0])
finance_df1 = pd.merge(finance_df1, period, on="ticker", how="left")
finance_df2 = pd.merge(finance_df2, period, on="ticker", how="left")

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
#データを結合
finance_df1["flag"] = 1; finance_df2["flag"] = 0
finance_df = pd.concat((finance_df1[["ticker", "company", "industry", "accounting", "single", "period", "year", "quarter", 
                                     "sales", "operating_income", "net_income", "flag"]],
                        finance_df2[["ticker", "company", "industry", "accounting", "single", "period", "year", "quarter", 
                                     "sales", "operating_income", "net_income", "flag"]]), axis=0)
finance_df.index = np.arange(finance_df.shape[0])
industry = finance_df[["ticker", "industry"]].iloc[np.where(finance_df["ticker"].duplicated()==False)[0]]
finance_df["industry"] = pd.merge(finance_df[["ticker"]], industry, on="ticker", how="left")["industry"]
finance_df.index = np.arange(finance_df.shape[0])
industry = finance_df[["ticker", "industry"]].iloc[np.where(finance_df["ticker"].duplicated()==False)[0]]
finance_df["industry"] = pd.merge(finance_df[["ticker"]], industry, on="ticker", how="left")["industry"]
company_name = finance_df[["ticker", "company"]].iloc[np.where(finance_df["ticker"].duplicated()==False)[0]]
finance_df["company"] = pd.merge(finance_df[["ticker"]], company_name, on="ticker", how="left")["company"]
finance_df = finance_df.sort_values(by=["ticker", "year", "quarter", "flag"])
finance_df.index = np.arange(finance_df.shape[0])

In [12]:
#重複を削除
finance_df = finance_df.iloc[np.where(finance_df[["ticker", "single", "year", "quarter"]].duplicated()==False)[0]]
finance_df.index = np.arange(finance_df.shape[0])

In [13]:
#データの書き出し
finance_df.to_csv("D:/Statistics/data/finance_data/finance_panel_data.csv", index=None)